# 1. Import Dependencies

In [ ]:
!pip install stable-baselines3[extra]

In [1]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [2]:
env_name = 'CartPole-v0'
env  = gym.make(env_name)

In [3]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:17.0
Episode:2 Score:37.0
Episode:3 Score:21.0
Episode:4 Score:13.0
Episode:5 Score:19.0


## Understanding the environment

In [4]:
env.action_space.sample()

1

In [5]:
env.observation_space.sample()

array([-2.5673613e-01,  1.7932388e+38, -1.4885615e-01,  2.7743178e+38],
      dtype=float32)

# 3. Train the RL Model
https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html

In [6]:
log_path = os.path.join('Training', 'Logs')

In [7]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [8]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1582 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1127       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00865233 |
|    clip_fraction        | 0.106      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | -0.00507   |
|    learning_rate        | 0.0003     |
|    loss                 | 6.41       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0169    |
|    value_loss           | 54.6       |
----------------------------------------
---------------------

# 4. Save and Reload Model

In [10]:
PPO_Path =os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [11]:
model.save(PPO_Path)

In [15]:
del model

In [19]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1755 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1173         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0052210772 |
|    clip_fraction        | 0.0173       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.532       |
|    explained_variance   | -0.0133      |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0199       |
|    n_updates            | 210          |
|    policy_gradient_loss | -0.000963    |
|    value_loss           | 0.113        |
----------------------------

In [17]:
model = PPO.load(PPO_Path, env=env)

In [18]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1833 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
--------------------------------------
| time/                   |          |
|    fps                  | 1195     |
|    iterations           | 2        |
|    time_elapsed         | 3        |
|    total_timesteps      | 4096     |
| train/                  |          |
|    approx_kl            | 0.009778 |
|    clip_fraction        | 0.0554   |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.537   |
|    explained_variance   | 0.661    |
|    learning_rate        | 0.0003   |
|    loss                 | 2.61     |
|    n_updates            | 110      |
|    policy_gradient_loss | -0.00557 |
|    value_loss           | 32.1     |
--------------------------------------
-----------------------------------------
| time/        

# 5. Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [22]:
env.close()

# 6. Testing

In [25]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


# 7. Viewing Tensorboard Logs

In [26]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [27]:
!tensorboard --logdir={training_log_path}

^C


# 8. Adding Callback

In [28]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [29]:
save_path = os.path.join('Training', 'Saved Models')

In [31]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [32]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [33]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 1889 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1224        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008394377 |
|    clip_fraction        | 0.117       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00455    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.74        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.019      |
|    value_loss           | 49.2        |
-----------------------------------------
---

In [34]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [35]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [38]:
env.close()

# 9. Change Policy

In [37]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [39]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [40]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 734  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 405         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014801528 |
|    clip_fraction        | 0.223       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00231    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.28        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0246     |
|    value_loss           | 17.3        |
-----------------------------------------
----------------------------------

# 10. Alt Algorithm

In [41]:
from stable_baselines3 import DQN

In [42]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [43]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.936    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8303     |
|    time_elapsed     | 0        |
|    total_timesteps  | 134      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.9      |
| time/               |          |
|    episodes         | 8        |
|    fps              | 8700     |
|    time_elapsed     | 0        |
|    total_timesteps  | 210      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.861    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 9085     |
|    time_elapsed     | 0        |
|    total_timesteps  | 292      |
----------------------------------
------------------------

In [44]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [45]:
model.save(dqn_path)

In [46]:
model = DQN.load(dqn_path, env=env)

In [47]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.4, 0.66332495807108)

In [48]:
env.close()